In [1]:

import tensorflow as tf
import os

train_dir='../dataset\\cats_and_dogs_small\\train'
validation_dir='../dataset\\cats_and_dogs_small\\validation'
test_dir='../dataset\\cats_and_dogs_small\\test'



train_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

# 검증 데이터는 argumentation 금지
test_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지의 크기를 150 × 150로 변경합니다
        target_size=(150, 150),
        batch_size=20,
        # binary_crossentropy 손실을 사용하므로 이진 레이블이 필요합니다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../dataset\\cats_and_dogs_small\\train'

In [2]:

## back born
conv_base=tf.keras.applications.VGG16(
                  weights='imagenet',  ## 모델을 포기화할 checkpoint 지정 (여기는 imagenet 확습 모델)
                  include_top=False   ## 네트워크 최상위 FC(분류기)를 포함할지 안할지 지정 (imagenet은 1000개 클래스이므로 지금 개 고양이 에선 사용x)
                  )  ## 네트워크에 입력할 이미지 텐서 크기
## 모델 구성 확인하기.
## 확인하면 최종출력은  (4,4,512)이다. 즉, 4x4x512=8192인 (batch,8192) 인 FC를 만들어 연결해주어야한다. 그러나 fc만 할거면 Flatten()해서 입력
## 이제 이거를 FC에 연결하여 학습을 하여 개 고양이 분류 모델을 완성한다.
conv_base.summary()

## 모델
input_Layer = tf.keras.layers.Input(shape=(150,150,3))
x=conv_base(input_Layer)
x=tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(512, activation='relu')(x)
Out_Layer= tf.keras.layers.Dense(1, activation='sigmoid')(x)

conv_base.trainable = True

print(conv_base.layers)
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':  ## block5_conv1이후의 모든 레이어들은 학습이 된다.
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:

loss_function=tf.keras.losses.binary_crossentropy
optimize=tf.keras.optimizers.RMSprop(lr=0.0001)
metric=tf.keras.metrics.binary_accuracy
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)